<a href="https://colab.research.google.com/github/DVK-MSU/testing/blob/main/spark_session_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

import numpy as np
import pandas as pd
#Функция для передачи результата sql-запроса в pandas DataFrame.
#Напиши select = """ SELECT * FROM table""" и вызови функцию sql(select)
def sql(select):
    SQLdf = spark.sql(select)
    return SQLdf.toPandas()

import pyspark.sql.functions as F

from google.colab import data_table
data_table.disable_dataframe_formatter()

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=54e991a0c98a5c49feae0a86acffa8359c2e7d2aa7b383aeedcc6730a61bcc9f
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


# Load data

In [5]:
def reheadering_ETL():
    #extraction data from csv
    path = "/content/drive/MyDrive/Spark_training/california_housing_train.csv"
    sc = spark.read.csv(path)
    sc.createOrReplaceTempView("df")
    #transformation data (reheadering columns)
    sc_pandas = sc.toPandas()
    columns_name = dict(zip(sc_pandas.columns.to_list(),sc_pandas.iloc[0].to_list()))
    sc_pandas.rename(columns=columns_name, inplace=True)
    sc_pandas.drop([0], inplace=True)
    sc = spark.createDataFrame(sc_pandas)
    #loading data to json
    sc.write.json('/content/drive/MyDrive/Spark_training/output', mode='overwrite')
reheadering_ETL()


In [6]:
spark.read.json('/content/drive/MyDrive/Spark_training/output').show() # output data

+-----------+------------------+---------+-----------+------------------+-------------+-----------+--------------+-----------+
| households|housing_median_age| latitude|  longitude|median_house_value|median_income| population|total_bedrooms|total_rooms|
+-----------+------------------+---------+-----------+------------------+-------------+-----------+--------------+-----------+
| 472.000000|         15.000000|34.190000|-114.310000|      66900.000000|     1.493600|1015.000000|   1283.000000|5612.000000|
| 463.000000|         19.000000|34.400000|-114.470000|      80100.000000|     1.820000|1129.000000|   1901.000000|7650.000000|
| 117.000000|         17.000000|33.690000|-114.560000|      85700.000000|     1.650900| 333.000000|    174.000000| 720.000000|
| 226.000000|         14.000000|33.640000|-114.570000|      73400.000000|     3.191700| 515.000000|    337.000000|1501.000000|
| 262.000000|         20.000000|33.570000|-114.570000|      65500.000000|     1.925000| 624.000000|    326.0000

In [7]:
sc_renamed = spark.read.json('/content/drive/MyDrive/Spark_training/output')
sc_renamed.dtypes


[('households', 'string'),
 ('housing_median_age', 'string'),
 ('latitude', 'string'),
 ('longitude', 'string'),
 ('median_house_value', 'string'),
 ('median_income', 'string'),
 ('population', 'string'),
 ('total_bedrooms', 'string'),
 ('total_rooms', 'string')]

In [8]:
def FloatType_Transform(sc_renamed):
    #Extract
    sc_renamed = spark.read.json('/content/drive/MyDrive/Spark_training/output')
    #Transform
    from pyspark.sql.functions import col
    return sc_renamed.select([col(col_name).cast('float') for col_name in sc_renamed.columns])
sc_renamed= sc_renamed.transform(FloatType_Transform)
sc_renamed.write.json('/content/drive/MyDrive/Spark_training/output/float_type_transform', mode='overwrite')

In [9]:
spark.read.json('/content/drive/MyDrive/Spark_training/output/float_type_transform').dtypes

[('households', 'double'),
 ('housing_median_age', 'double'),
 ('latitude', 'double'),
 ('longitude', 'double'),
 ('median_house_value', 'double'),
 ('median_income', 'double'),
 ('population', 'double'),
 ('total_bedrooms', 'double'),
 ('total_rooms', 'double')]

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Frame Mappers

Получение нового DF с преобразованными значениями

In [15]:
sc_renamed.show()

+----------+------------------+--------+---------+------------------+-------------+----------+--------------+-----------+
|households|housing_median_age|latitude|longitude|median_house_value|median_income|population|total_bedrooms|total_rooms|
+----------+------------------+--------+---------+------------------+-------------+----------+--------------+-----------+
|     472.0|              15.0|   34.19|  -114.31|           66900.0|       1.4936|    1015.0|        1283.0|     5612.0|
|     463.0|              19.0|    34.4|  -114.47|           80100.0|         1.82|    1129.0|        1901.0|     7650.0|
|     117.0|              17.0|   33.69|  -114.56|           85700.0|       1.6509|     333.0|         174.0|      720.0|
|     226.0|              14.0|   33.64|  -114.57|           73400.0|       3.1917|     515.0|         337.0|     1501.0|
|     262.0|              20.0|   33.57|  -114.57|           65500.0|        1.925|     624.0|         326.0|     1454.0|
|     239.0|            

In [36]:
df_1 = sc_renamed.rdd.map(lambda x: (x['households']/10, x['housing_median_age']/2))\
.toDF(['households','housing_median_age'])

In [37]:
df_1.show()

+----------+------------------+
|households|housing_median_age|
+----------+------------------+
|      47.2|               7.5|
|      46.3|               9.5|
|      11.7|               8.5|
|      22.6|               7.0|
|      26.2|              10.0|
|      23.9|              14.5|
|      63.3|              12.5|
|      15.8|              20.5|
|     105.6|              17.0|
|      27.1|              23.0|
|      82.4|               8.0|
|      43.7|              10.5|
|      21.1|              24.0|
|      47.9|              15.5|
|      30.0|               7.5|
|      40.1|               8.5|
|      25.6|              14.0|
|       2.7|              10.5|
|      32.0|               8.5|
|       1.5|               8.5|
+----------+------------------+
only showing top 20 rows



Добавление новой колноки, рассчитаной на основе существующих

In [40]:
from pyspark.sql.functions import lit

In [52]:
df_2 = df_1.withColumn('multiplication_col', df_1.households*df_1.housing_median_age)

In [59]:
df_2.show()

+----------+------------------+------------------+
|households|housing_median_age|multiplication_col|
+----------+------------------+------------------+
|      47.2|               7.5|             354.0|
|      46.3|               9.5|439.84999999999997|
|      11.7|               8.5| 99.44999999999999|
|      22.6|               7.0|158.20000000000002|
|      26.2|              10.0|             262.0|
|      23.9|              14.5|346.54999999999995|
|      63.3|              12.5|            791.25|
|      15.8|              20.5|323.90000000000003|
|     105.6|              17.0|1795.1999999999998|
|      27.1|              23.0| 623.3000000000001|
|      82.4|               8.0|             659.2|
|      43.7|              10.5|            458.85|
|      21.1|              24.0|506.40000000000003|
|      47.9|              15.5| 742.4499999999999|
|      30.0|               7.5|             225.0|
|      40.1|               8.5|            340.85|
|      25.6|              14.0|

In [68]:
from pyspark.sql.functions import udf

In [69]:
@udf
def rounding(x):
    return round(x, 2)
# rounding_udf = udf(rounding)

df_3 = df_2.select(df_2.households, df_2.housing_median_age, rounding(df_2.multiplication_col))


In [70]:
df_3.show()

+----------+------------------+----------------------------+
|households|housing_median_age|rounding(multiplication_col)|
+----------+------------------+----------------------------+
|      47.2|               7.5|                       354.0|
|      46.3|               9.5|                      439.85|
|      11.7|               8.5|                       99.45|
|      22.6|               7.0|                       158.2|
|      26.2|              10.0|                       262.0|
|      23.9|              14.5|                      346.55|
|      63.3|              12.5|                      791.25|
|      15.8|              20.5|                       323.9|
|     105.6|              17.0|                      1795.2|
|      27.1|              23.0|                       623.3|
|      82.4|               8.0|                       659.2|
|      43.7|              10.5|                      458.85|
|      21.1|              24.0|                       506.4|
|      47.9|            

In [71]:
df_3 = df_3.withColumnRenamed('rounding(multiplication_col)', 'multiplication_col')

In [72]:
df_3.show()

+----------+------------------+------------------+
|households|housing_median_age|multiplication_col|
+----------+------------------+------------------+
|      47.2|               7.5|             354.0|
|      46.3|               9.5|            439.85|
|      11.7|               8.5|             99.45|
|      22.6|               7.0|             158.2|
|      26.2|              10.0|             262.0|
|      23.9|              14.5|            346.55|
|      63.3|              12.5|            791.25|
|      15.8|              20.5|             323.9|
|     105.6|              17.0|            1795.2|
|      27.1|              23.0|             623.3|
|      82.4|               8.0|             659.2|
|      43.7|              10.5|            458.85|
|      21.1|              24.0|             506.4|
|      47.9|              15.5|            742.45|
|      30.0|               7.5|             225.0|
|      40.1|               8.5|            340.85|
|      25.6|              14.0|